In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,27853
2,Huelva,Confirmados PDIA 14 días,1226
3,Huelva,Tasa PDIA 14 días,"238,90718475359043"
4,Huelva,Confirmados PDIA 7 días,571
5,Huelva,Total Confirmados,28026
6,Huelva,Curados,23464
7,Huelva,Fallecidos,362


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  27853.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  7227.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1081 personas en los últimos 7 días 

Un positivo PCR cada 521 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,27853.0,571.0,1226.0,513170.0,111.269170,238.907185,53.0
Huelva-Costa,16346.0,215.0,487.0,289548.0,74.253664,168.193184,28.0
Condado-Campiña,8611.0,290.0,621.0,156231.0,185.622572,397.488335,24.0
Huelva (capital),7227.0,133.0,276.0,143837.0,92.465777,191.883869,12.0
Sierra de Huelva-Andévalo Central,2487.0,59.0,101.0,67391.0,87.548782,149.871645,8.0
Cartaya,1239.0,38.0,126.0,20083.0,189.214759,627.396305,5.0
Bollullos Par del Condado,764.0,32.0,55.0,14387.0,222.423021,382.289567,4.0
Isla Cristina,2032.0,11.0,13.0,21393.0,51.418688,60.767541,4.0
Moguer,1049.0,41.0,87.0,21867.0,187.497142,397.859789,3.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cumbres Mayores,43.0,18.0,18.0,1749.0,1029.159520,1029.159520,0.0
Almonte,1418.0,106.0,237.0,24507.0,432.529481,967.070633,3.0
Santa Ana la Real,9.0,3.0,4.0,475.0,631.578947,842.105263,0.0
Rociana del Condado,491.0,30.0,66.0,7939.0,377.881345,831.338960,2.0
Cumbres de San Bartolomé,8.0,3.0,3.0,376.0,797.872340,797.872340,0.0
Cartaya,1239.0,38.0,126.0,20083.0,189.214759,627.396305,5.0
Galaroza,24.0,8.0,8.0,1382.0,578.871201,578.871201,0.0
Almonaster la Real,63.0,5.0,9.0,1788.0,279.642058,503.355705,0.0
San Bartolomé de la Torre,282.0,6.0,15.0,3761.0,159.532039,398.830098,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Minas de Riotinto,107.0,1.0,7.0,3812.0,26.232949,183.630640,0.0,0.142857
Palos de la Frontera,565.0,7.0,43.0,11742.0,59.615057,366.206779,2.0,0.162791
Ayamonte,1262.0,1.0,6.0,21104.0,4.738438,28.430629,0.0,0.166667
Cartaya,1239.0,38.0,126.0,20083.0,189.214759,627.396305,5.0,0.301587
Aroche,88.0,1.0,3.0,3054.0,32.743942,98.231827,0.0,0.333333
Lepe,2064.0,9.0,24.0,27880.0,32.281205,86.083214,3.0,0.375000
Beas,178.0,5.0,13.0,4341.0,115.180834,299.470168,3.0,0.384615
San Bartolomé de la Torre,282.0,6.0,15.0,3761.0,159.532039,398.830098,0.0,0.400000
Aracena,505.0,8.0,20.0,8255.0,96.910963,242.277408,3.0,0.400000
